In [15]:
import pandas as pd
import numpy as np
from gurobipy import *

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from scipy.optimize import linprog

In [16]:
X_separable = np.array([[-2],[-1],[1],[2]])
y_separable = np.array([[-1],[-1],[1],[1]])
# If the X values are plotted on the X axis, there will be clear linear separation between label -1 and 1
# In this case the LP will give an optimal solution
# Indicates the MLE does not Exist

In [17]:
X_nonseparable = np.array([[-2],[-1],[1],[2]])
y_nonseparable = np.array([[-1],[1],[-1],[1]])
# If the X values are plotted on the X axis, there are no linear separation between label -1 and 1
# In this case the LP will say Infeasible solution
# Indicates the MLE Exists

# Separable

In [18]:
X = pd.DataFrame(X_separable)
X

,0
0,-2
1,-1
2,1
3,2


In [19]:
y= pd.DataFrame(y_separable)
y

,0
0,-1
1,-1
2,1
3,1


# Scipy Way (Separable)

In [20]:
tmp = X.values
tmp = sc.fit_transform(tmp)


xx = np.array(y.values.reshape(-1,1) * tmp)
t = y.values

A_ub = np.append(xx, t.reshape(-1,1), 1)
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)

c_obj = np.repeat(1, A_ub.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

     con: array([], dtype=float64)
     fun: 0.4367339119588688
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.056134  , -0.91850518, -1.35675246, -1.49438128])
  status: 2
 success: False
       x: array([0.21761027, 0.21912364])

# Gurobi Way (Separable)

In [21]:
n,p = X.shape
y = np.squeeze(y)
m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]
m.update()


float32_epsilon = (np.finfo(np.float32).eps)*10 

# Constraints
for i in range(n):
    m.addConstr(y[i]*(X.iloc[i].dot(v))>= float32_epsilon)
m.update()


# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 1 columns and 4 nonzeros
Model fingerprint: 0x53da884e
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
Model Status: Optimal or Others


In [24]:
for v in m.getVars():
        print('%s %s %g' % (v.varName,':', v.x))

v0 : 1.19209e-06


# Inseparable

In [26]:
X = pd.DataFrame(X_nonseparable)
X

,0
0,-2
1,-1
2,1
3,2


In [27]:
y = pd.DataFrame(y_nonseparable)
y

,0
0,-1
1,1
2,-1
3,1


# Scipy Way (Inseparable)

In [29]:
tmp = X.values
tmp = sc.fit_transform(tmp)


xx = np.array(y.values.reshape(-1,1) * tmp)
t = y.values

A_ub = np.append(xx, t.reshape(-1,1), 1)
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)

c_obj = np.repeat(1, A_ub.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

     con: array([], dtype=float64)
     fun: 3.3294229710976895
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.91161049, -1.272659  ,  1.64119799, -4.82546748])
  status: 2
 success: False
       x: array([1.87249448, 1.45692849])

# Gurobi Way (Inseparable)

In [30]:
n,p = X.shape
y = np.squeeze(y)
m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]
m.update()


float32_epsilon = (np.finfo(np.float32).eps)*10 

# Constraints
for i in range(n):
    m.addConstr(y[i]*(X.iloc[i].dot(v))>= float32_epsilon)
m.update()


# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 1 columns and 4 nonzeros
Model fingerprint: 0xd85ed01f
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds
Infeasible model
Model Status: Infeasible


# Break Down of Codes in Scipy way

In [31]:
X = pd.DataFrame(X_separable)
X

,0
0,-2
1,-1
2,1
3,2


In [32]:
y= pd.DataFrame(y_separable)
y

,0
0,-1
1,-1
2,1
3,1


In [33]:
tmp = X.values
tmp

array([[-2],
       [-1],
       [ 1],
       [ 2]])

In [34]:
tmp = sc.fit_transform(tmp)
tmp

array([[-1.26491106],
       [-0.63245553],
       [ 0.63245553],
       [ 1.26491106]])

In [35]:
xx = np.array(y.values.reshape(-1,1) * tmp)
xx

array([[1.26491106],
       [0.63245553],
       [0.63245553],
       [1.26491106]])

In [36]:
t = y.values
t

array([[-1],
       [-1],
       [ 1],
       [ 1]])

In [37]:
A_ub = np.append(xx, t.reshape(-1,1), 1)
A_ub

array([[ 1.26491106, -1.        ],
       [ 0.63245553, -1.        ],
       [ 0.63245553,  1.        ],
       [ 1.26491106,  1.        ]])

In [38]:
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)
b_ub

array([[-1],
       [-1],
       [-1],
       [-1]])

In [39]:
c_obj = np.repeat(1, A_ub.shape[1])
c_obj

array([1, 1])

In [41]:
res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":1000})
res

     con: array([], dtype=float64)
     fun: 0.4367339119588688
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.056134  , -0.91850518, -1.35675246, -1.49438128])
  status: 2
 success: False
       x: array([0.21761027, 0.21912364])